In [2]:
%load_ext autoreload
%autoreload 2

In [218]:
import tensorflow as tf
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, accuracy_score

from utils import load_vectors, rnd_labeled_data_generator, test_data_generator, conseq_labeled_data_generator

Загружаем векторы

In [4]:
vectors = {
    "auto": "../data/auto_vectors.csv",
    "mus": "../data/mus_vectors.csv"
}

In [5]:
%%time
auto_df = load_vectors(vectors["auto"], manually=True)

'Reading line 20473'

Wall time: 7min 11s


In [204]:
auto_df.groupby("overall").count()

,vectors,target
overall,,
1.0,542,542
2.0,606,606
3.0,1430,1430
4.0,3967,3967
5.0,13928,13928


Классификация с 5 категориями, не бинарная

In [8]:
auto_df["target"] = pd.get_dummies(auto_df["overall"]).values.tolist()
auto_df["target"] = auto_df["target"].apply(np.array)

In [9]:
auto_df.head()

,overall,vectors,target
0,5.0,"[[-0.67167366, 1.4340448, 0.70772099, 0.267104...","[0, 0, 0, 0, 1]"
1,4.0,"[[-3.4646065, 0.90924335, 0.051792312, -0.0250...","[0, 0, 0, 1, 0]"
2,5.0,"[[-0.34402627, -1.2019368, 0.49981338, -1.2384...","[0, 0, 0, 0, 1]"
3,5.0,"[[-1.187706, 0.5451839, 1.5461149, -0.60059845...","[0, 0, 0, 0, 1]"
4,5.0,"[[-0.78727818, 1.2093724, -2.3347111, -2.42030...","[0, 0, 0, 0, 1]"


Если делать zero-padding, очень быстро возникает MemoryError. Лучше учить по одному сэмплу

In [5]:
def max_len_padding(list_of_matrices):
    pairwise_len_max = lambda a, b: max(len(a), len(b))
    max_len = max([i.shape[0] for i in list_of_matrices])
    return max_len

def pad_by_zero_seqs_generator(list_of_matrices, y_list, timesteps, batch_size):
#     max_len = timesteps
#     aligned = [np.pad(i, ((0, max_len - i.shape[0]), (0, 0)), "constant", constant_values=0.0) for i in list_of_matrices]
#     aligned = np.array(aligned)
    while True:
        indices = np.random.choice(np.arange(len(list_of_matrices)), size=[batch_size])
        x = np.array(list_of_matrices)[indices]
        x = [np.pad(i, ((0, timesteps - i.shape[0]), (0, 0)), "constant", constant_values=0.0) for i in x]
        x = np.array(x)
        y = np.array(y_list)[indices].reshape([1, 1, 1])
        yield x, y

In [136]:
X_train, X_test, y_train, y_test = train_test_split(auto_df["vectors"].values, auto_df["target"].values, test_size=0.3)

In [137]:
y_test = np.dstack(y_test)[0].T
y_train = np.dstack(y_train)[0].T

In [33]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [288]:
# batch_size = 100
# timesteps = timesteps_padding(auto_df["vectors"].values)
# print("Timesteps:", timesteps)
# print("Batch size:", batch_size)
hidden_size1 = 80
hidden_size2 = 32

model = Sequential()
model.add(LSTM(hidden_size1, return_sequences=True, input_shape=(None, 128)))
# model.add(LSTM(hidden_size2, return_sequences=False))
model.add(Dense(5, activation="sigmoid"))
# model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='sgd', metrics=["accuracy"])

model.fit_generator(rnd_labeled_data_generator(X_train, y_train), 
                    validation_data=rnd_labeled_data_generator(X_test, y_test), 
                    steps_per_epoch=30, epochs=150, verbose=1, validation_steps=100)

Epoch 1/150
30/30 [==============================] - 10s 333ms/step - loss: 0.7199 - acc: 0.4738 - val_loss: 0.6901 - val_acc: 0.5435
Epoch 2/150
30/30 [==============================] - 1s 44ms/step - loss: 0.6796 - acc: 0.5809 - val_loss: 0.6700 - val_acc: 0.5939
Epoch 3/150
30/30 [==============================] - 2s 59ms/step - loss: 0.6553 - acc: 0.6412 - val_loss: 0.6400 - val_acc: 0.6688
Epoch 4/150
30/30 [==============================] - 2s 66ms/step - loss: 0.6339 - acc: 0.6853 - val_loss: 0.6242 - val_acc: 0.7044
Epoch 5/150
30/30 [==============================] - 2s 65ms/step - loss: 0.6150 - acc: 0.7298 - val_loss: 0.6061 - val_acc: 0.7472
Epoch 6/150
30/30 [==============================] - 1s 36ms/step - loss: 0.5901 - acc: 0.7802 - val_loss: 0.5869 - val_acc: 0.7797
Epoch 7/150
30/30 [==============================] - 1s 47ms/step - loss: 0.5514 - acc: 0.8308 - val_loss: 0.5737 - val_acc: 0.7894
Epoch 8/150
30/30 [==============================] - 2s 71ms/step - loss: 

30/30 [==============================] - 1s 41ms/step - loss: 0.3297 - acc: 0.8922 - val_loss: 0.3569 - val_acc: 0.8695
Epoch 63/150
30/30 [==============================] - 1s 39ms/step - loss: 0.3719 - acc: 0.8644 - val_loss: 0.3364 - val_acc: 0.8807
Epoch 64/150
30/30 [==============================] - 1s 33ms/step - loss: 0.3293 - acc: 0.8810 - val_loss: 0.3290 - val_acc: 0.8791
Epoch 65/150
30/30 [==============================] - 1s 36ms/step - loss: 0.2921 - acc: 0.9055 - val_loss: 0.3686 - val_acc: 0.8557
Epoch 66/150
30/30 [==============================] - 1s 33ms/step - loss: 0.3307 - acc: 0.8908 - val_loss: 0.3511 - val_acc: 0.8690
Epoch 67/150
30/30 [==============================] - 1s 31ms/step - loss: 0.3495 - acc: 0.8790 - val_loss: 0.3429 - val_acc: 0.8834
Epoch 68/150
30/30 [==============================] - 1s 35ms/step - loss: 0.4012 - acc: 0.8275 - val_loss: 0.3530 - val_acc: 0.8565
Epoch 69/150
30/30 [==============================] - 1s 31ms/step - loss: 0.3458 

30/30 [==============================] - 1s 37ms/step - loss: 0.3981 - acc: 0.7957 - val_loss: 0.3619 - val_acc: 0.8521
Epoch 124/150
30/30 [==============================] - 1s 38ms/step - loss: 0.3340 - acc: 0.8783 - val_loss: 0.2979 - val_acc: 0.8937
Epoch 125/150
30/30 [==============================] - 1s 33ms/step - loss: 0.2608 - acc: 0.9186 - val_loss: 0.2905 - val_acc: 0.8963
Epoch 126/150
30/30 [==============================] - 1s 29ms/step - loss: 0.3473 - acc: 0.8788 - val_loss: 0.3424 - val_acc: 0.8587
Epoch 127/150
30/30 [==============================] - 1s 31ms/step - loss: 0.2985 - acc: 0.8941 - val_loss: 0.2890 - val_acc: 0.9074
Epoch 128/150
30/30 [==============================] - 1s 29ms/step - loss: 0.3941 - acc: 0.8427 - val_loss: 0.3954 - val_acc: 0.8223
Epoch 129/150
30/30 [==============================] - 1s 27ms/step - loss: 0.3479 - acc: 0.8662 - val_loss: 0.3347 - val_acc: 0.8623
Epoch 130/150
30/30 [==============================] - 1s 32ms/step - loss: 

In [220]:
def predict(X, model):
    predictions = []
    for i, sample in enumerate(X):
        clear_output(True)
        display("Retrieving {}/{}".format(i + 1, X.shape[0]))
        predictions.append(model.predict(np.reshape(sample, [1, *sample.shape]), steps=1).reshape(-1, 5))
#     predictions = np.array(predictions)
#     predictions = np.reshape(predictions, (predictions.shape[0]))
    return predictions
    

model.predict() автоматически пытается склеить последовательности разной длины в один 2D-массив и ломается

In [276]:
def result_to_one_hot(prediction):
    # Усреднение ответов для каждого слова внутри текста, потом выбор максимума
#     max_i = np.argmax(np.mean(prediction, axis=0).reshape(-1))
#     res = np.zeros(5)
#     res[max_i] = 1
#     return res.astype(int)
    # Выбор оценки исходя из самого значимого слова
#     max_i = np.argmax(np.max(prediction, axis=0))
#     res = np.zeros(5)
#     res[max_i] = 1
#     return res.astype(int)
    # Выбор по количеству максимальных
    # значений среди слов в пределах рубрики
    max_i = np.argmax(np.argmax(prediction, axis=0))
    res = np.zeros(5)
    res[max_i] = 1
    return res.astype(int)


def decode_one_hot(oh_matrix):
    return np.where(oh_matrix == 1)[1] + 1

In [289]:
res = predict(X_test, model)

'Retrieving 6142/6142'

In [290]:
oh_res = np.array(list(map(result_to_one_hot, res)))

In [291]:
print("Balanced accuracy:\t\t{}".format(balanced_accuracy_score(decode_one_hot(y_test), 
                                                                         decode_one_hot(oh_res), adjusted=False)))
print("Balanced and adjusted accuracy:\t{}".format(balanced_accuracy_score(decode_one_hot(y_test), 
                                                                         decode_one_hot(oh_res), adjusted=True)))
print("Unbalanced accuracy:\t\t{}".format(accuracy_score(decode_one_hot(y_test), 
                                                         decode_one_hot(oh_res))))

Balanced accuracy:		0.21376515198416973
Balanced and adjusted accuracy:	0.01720643998021215
Unbalanced accuracy:		0.554379680885705


In [292]:
decode_one_hot(y_test[:10])

array([5, 5, 5, 5, 5, 5, 4, 4, 4, 5], dtype=int64)

In [293]:
decode_one_hot(oh_res[:10])

array([5, 5, 4, 2, 3, 5, 5, 1, 5, 5], dtype=int64)

In [294]:
oh_res[:10]

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1]])